# ACTIVIDAD : Redes Convolucionales

---

En esta actividad, vamos a trabajar con Convolutional Neural Networks para resolver un problema de clasificación de imágenes. En particular, vamos a clasificar imágenes de personajes los Simpsons.

<center><img src="https://i.imgur.com/i8zIGqX.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consiste en imágenes de personajes de la serie extraídas directamente de capítulos de la serie. Este dataset ha sido recopilado por [Alexandre Attia](http://www.alexattia.fr/)
Partiendo de los 18 personajes etiquetados, éstos pueden aparecer en distintas poses, en distintas posiciones de la imagen o con otros personajes en pantalla (si bien el personaje a clasificar siempre aparece en la posición predominante).

El dataset de training puede ser descargado desde aquí:

[Training data](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219337&authkey=AMzI92bJPx8Sd60) (~500MB)

Por otro lado, el dataset de test puede ser descargado de aquí:

[Test data](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219341&authkey=ANnjK3Uq1FhuAe8) (~10MB)

Antes de empezar la práctica, se recomienda descargar las imágenes y echarlas un vistazo.


## Carga de los datos

In [1]:
## Librerías utilizadas
import cv2
import os
import numpy as np
import keras
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
import kagglehub
import shutil
import random
from sklearn.model_selection import train_test_split

c:\Users\sergi.zarzuelo.abel1\AppData\Local\miniconda3\envs\no_estr_fed2_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Nos descargamos la carpeta de la web de Kaggle:
https://www.kaggle.com/datasets/alexattia/the-simpsons-characters-dataset?resource=download&select=simpsons_dataset

In [2]:
## Descargar dataset
path = kagglehub.dataset_download("alexattia/the-simpsons-characters-dataset")
print("Descargado en:", path)

## Definir rutas
source_path = os.path.join(path, "simpsons_dataset")
target_path = "./simpsons_dataset"

## Si ya existe, eliminar y copiar de nuevo
if os.path.exists(target_path):
    shutil.rmtree(target_path)
    print(f"⚠️ Carpeta existente eliminada: {target_path}")

shutil.copytree(source_path, target_path)
print(f"✅ Dataset copiado a: {target_path}")

Descargado en: C:\Users\sergi.zarzuelo.abel1\.cache\kagglehub\datasets\alexattia\the-simpsons-characters-dataset\versions\4
⚠️ Carpeta existente eliminada: ./simpsons_dataset
✅ Dataset copiado a: ./simpsons_dataset


Podemos ver que los personajes están repartidos en carpetas

In [3]:
# ## Los personajes se encuentran en carpetas
# !ls $path/simpsons_dataset


In [4]:
dataset_dir = target_path
conteo_imagenes = {}

for cls in sorted(os.listdir(dataset_dir)):
    cls_path = os.path.join(dataset_dir, cls)
    if os.path.isdir(cls_path):
        imagenes = [f for f in os.listdir(cls_path) if f.endswith('.jpg')]
        conteo_imagenes[cls] = len(imagenes)

## Mostrar resultados ordenados
for cls, count in sorted(conteo_imagenes.items(), key=lambda x: x[1]):
    print(f"{cls:30s} -> {count} imágenes")

simpsons_dataset               -> 0 imágenes
lionel_hutz                    -> 3 imágenes
disco_stu                      -> 8 imágenes
troy_mcclure                   -> 8 imágenes
miss_hoover                    -> 17 imágenes
fat_tony                       -> 27 imágenes
gil                            -> 27 imágenes
otto_mann                      -> 32 imágenes
sideshow_mel                   -> 40 imágenes
agnes_skinner                  -> 42 imágenes
rainier_wolfcastle             -> 45 imágenes
cletus_spuckler                -> 47 imágenes
snake_jailbird                 -> 55 imágenes
professor_john_frink           -> 65 imágenes
martin_prince                  -> 71 imágenes
patty_bouvier                  -> 72 imágenes
ralph_wiggum                   -> 89 imágenes
carl_carlson                   -> 98 imágenes
selma_bouvier                  -> 103 imágenes
barney_gumble                  -> 106 imágenes
groundskeeper_willie           -> 121 imágenes
maggie_simpson                 -> 1

Como Lionel solo tiene tres imágenes, lo quitamos del entrenamiento

In [5]:
# !rm -r ./simpsons_dataset/lionel_hutz

In [6]:
import shutil

shutil.rmtree("./simpsons_dataset/lionel_hutz")

Ahora generaremos diferentes carpetas separando los datos de train, test y validación

In [7]:
## Definimos los paths
ORIGINAL_DATASET_DIR = './simpsons_dataset'
BASE_OUTPUT_DIR = './simpsons_split_dataset'

## Porcentajes; he optado por generar sets de train, test y validación.
## Pero podéis variar los porcentajes
train_pct = 0.7
val_pct = 0.15
test_pct = 0.15

## Crear estructura de carpetas
splits = ['train', 'val', 'test']
classes = os.listdir(ORIGINAL_DATASET_DIR)
classes = [cls for cls in classes if os.path.isdir(os.path.join(ORIGINAL_DATASET_DIR, cls))]

for split in splits:
    for cls in classes:
        os.makedirs(os.path.join(BASE_OUTPUT_DIR, split, cls), exist_ok=True)

## Dividir y copiar imágenes
for cls in classes:
    cls_path = os.path.join(ORIGINAL_DATASET_DIR, cls)
    images = [f for f in os.listdir(cls_path) if f.endswith('.jpg')]
    random.shuffle(images)

    # Asignación predeterminada
    train, val, test = [], [], []

    if len(images) >= 3:
        train, temp = train_test_split(images, train_size=train_pct, random_state=42)
        val, test = train_test_split(temp, test_size=test_pct / (test_pct + val_pct), random_state=42)
    else:
        # Si hay muy pocas imágenes, lo mandamos todo al entrenamiento
        train = images
        print(f"[Aviso] Clase '{cls}' tiene muy pocas imágenes ({len(images)}). Se asignan todas a entrenamiento.")

    for img_list, split in zip([train, val, test], ['train', 'val', 'test']):
        for img in img_list:
            src = os.path.join(cls_path, img)
            dst = os.path.join(BASE_OUTPUT_DIR, split, cls, img)
            shutil.copyfile(src, dst)

[Aviso] Clase 'simpsons_dataset' tiene muy pocas imágenes (0). Se asignan todas a entrenamiento.


In [8]:
## Crear diccionario de mapeo automáticamente
MAP_CHARACTERS = {i: cls for i, cls in enumerate(sorted(classes))}
MAP_CHARACTERS

{0: 'abraham_grampa_simpson',
 1: 'agnes_skinner',
 2: 'apu_nahasapeemapetilon',
 3: 'barney_gumble',
 4: 'bart_simpson',
 5: 'carl_carlson',
 6: 'charles_montgomery_burns',
 7: 'chief_wiggum',
 8: 'cletus_spuckler',
 9: 'comic_book_guy',
 10: 'disco_stu',
 11: 'edna_krabappel',
 12: 'fat_tony',
 13: 'gil',
 14: 'groundskeeper_willie',
 15: 'homer_simpson',
 16: 'kent_brockman',
 17: 'krusty_the_clown',
 18: 'lenny_leonard',
 19: 'lisa_simpson',
 20: 'maggie_simpson',
 21: 'marge_simpson',
 22: 'martin_prince',
 23: 'mayor_quimby',
 24: 'milhouse_van_houten',
 25: 'miss_hoover',
 26: 'moe_szyslak',
 27: 'ned_flanders',
 28: 'nelson_muntz',
 29: 'otto_mann',
 30: 'patty_bouvier',
 31: 'principal_skinner',
 32: 'professor_john_frink',
 33: 'rainier_wolfcastle',
 34: 'ralph_wiggum',
 35: 'selma_bouvier',
 36: 'sideshow_bob',
 37: 'sideshow_mel',
 38: 'simpsons_dataset',
 39: 'snake_jailbird',
 40: 'troy_mcclure',
 41: 'waylon_smithers'}

## Ejercicio

Utilizando Convolutional Neural Networks, entrenar al menos dos clasificadores que sean capaz de reconocer personajes en imágenes de los Simpsons con una accuracy en el dataset de test de, al menos, **90%**. Redactar un informe analizando varias de las alternativas probadas y los resultados obtenidos.

A continuación se detallan una serie de aspectos orientativos que podrían ser analizados en vuestro informe (no es necesario tratar todos ellos, pero cuánta más información podáis aportar mejor a la hora de desarrollar vuestro modelo):

*   Análisis de los datos a utilizar. ¿Qué distribución siguen? ¿Están las clases balanceadas?
*   Análisis de resultados, obtención de métricas de *precision* y *recall* por clase y análisis de qué clases obtienen mejores o peores resultados.
*   Análisis visual de los errores de la red. ¿Qué tipo de imágenes o qué personajes dan más problemas a nuestro modelo?
*   Comparación de modelos CNNs con un modelo de Fully Connected (sin convolución) para este problema.
*   Utilización de distintas arquitecturas CNNs, comentando aspectos como su profundidad, hiperparámetros utilizados, optimizador, uso de técnicas de regularización, *batch normalization*, etc.
*   Utilización de *data augmentation*. Esto puede conseguirse con la clase [ImageDataGenerator](https://keras.io/preprocessing/image/#imagedatagenerator-class) de Keras.


Notas:
* Los datos están en una única carpeta, por lo que tendrás que hacer el split entre train y test
* Recuerda partir los datos en training/validation para tener una buena estimación de los valores que nuestro modelo tendrá en los datos de test, así como comprobar que no estamos cayendo en overfitting. Una posible partición puede ser 80 / 20.
* No es necesario mostrar en el notebook las trazas de entrenamiento de todos los modelos entrenados, si bien una buena idea seria guardar gráficas de esos entrenamientos para el análisis. Sin embargo, **se debe mostrar el entrenamiento completo de al menos los dos mejores modelos obtenidos y la evaluación de los datos de test con estos modelos**.
* Las imágenes **no están normalizadas**. Hay que normalizarlas como hemos hecho en trabajos anteriores.
* El test set del problema tiene imágenes un poco más sencillas de identificar, por lo que es posible encontrarse con métricas en el test set bastante mejores que en el training set.

# Analisis de los datos

En esta sección se analizan los datos que se utilizarán en el entrenamiento de los modelos.

# Modelo 1 (CNN simple)

En esta sección se va a desarrollar un modelo CNN simple. Aunque Keras o TensorFlow pueden ser mas amigables a la hora de programar un modelo, este será desarrollado en PyTorch dado que es más complicado (estamos para aprender) y más utilizado en casos reales.

In [10]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 📍 Rutas a tus carpetas
train_dir = "./simpsons_split_dataset/train"
val_dir = "./simpsons_split_dataset/val"
test_dir = "./simpsons_split_dataset/test"

# 🔄 Transformaciones
transform_train = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

transform_val_test = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 📂 Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
val_dataset   = datasets.ImageFolder(val_dir, transform=transform_val_test)
test_dataset  = datasets.ImageFolder(test_dir, transform=transform_val_test)

# 🧾 Clases
print("Clases detectadas:", train_dataset.classes)

# 🧳 DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 👁 Visualización de imágenes (opcional)
def show_batch(loader, dataset):
    images, labels = next(iter(loader))
    fig, ax = plt.subplots(4, 8, figsize=(12, 6))
    for i in range(32):
        img = images[i].permute(1, 2, 0) * 0.5 + 0.5  # desnormaliza
        ax[i//8, i%8].imshow(img)
        ax[i//8, i%8].axis('off')
        ax[i//8, i%8].set_title(dataset.classes[labels[i]])
    plt.tight_layout()
    plt.show()

# (Opcional) Visualiza un batch de entrenamiento
show_batch(train_loader, train_dataset)

OSError: [WinError 182] The operating system cannot run %1. Error loading "c:\Users\sergi.zarzuelo.abel1\AppData\Local\miniconda3\envs\no_estr_fed2_env\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.